# Vector Representations of Words

이 튜토리얼에서는 [Mikolov et al.](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)의 word2vec 모델을 살펴보겠습니다. 이 모델은 "word embeddings"라 불리는 단어의 표현나타내는 벡터를 학습하는데 사용됩니다. 


## Highlights
이 튜토리얼은 TensorFlow에서 word2vec 모델 구축의 흥미롭고 중요한 부분을 강조합니다. 

* 먼저 동기부여로 단어를 벡터로 표현하려는 이유에 대해 알아보겠습니다.  
* 모델에 감춰진 훈련 방법에 대한 직관을 알아 보겠습니다(좋은 측정을 위한 수학적 접근). 
* TensorFlow에서 모델의 간단한 구현을 보여줍니다. 
* 마지막으로 최초 버전을 개선하는 방법을 살펴보겠습니다. 

튜토리얼에서는 나중에 코드를 다룹니다.하지만 직접 살펴보길 원한다면 다음 링크에서 기본적인 구현을 보기 바랍니다.[tensorflow/examples/tutorials/word2vec/word2vec_basic.py](https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/examples/tutorials/word2vec/word2vec_basic.py) 이 기본 예제에는 일부 데이터를 다운로드하고, 비트 맵을 작성하고 결과를 시각화하는 데 필요한 코드가 들어 있습니다. 기본 버전을 살펴보고, 실행하는게 익숙해지면 다음 코드로 완전히 이해할 수 있습니다. [tensorflow_models/tutorials/embedding/word2vec.py](https://www.github.com/tensorflow/tensorflow/blob/r1.1/tensorflow_models/tutorials/embedding/word2vec.py) 이는 텍스트 모델로 데이터를 이동하는데 스레드를 효율적으로 사용하는 방법, 학습 도중 checkpoint 설정 방법 등에 대한 보다 개선 된 TensorFlow 기능을 보여주는 심화구현입니다.

하지만, 우리가 왜 "word embeddings"을 배워야 하는지를 먼저 보겠습니다. 임베딩 전문가인 경우 세부 정보를 먼저 보고 싶다면로 본 섹션을 건너 뛰십시오.

## Motivation: Why Learn Word Embeddings?
 
이미지 처리 시스템은 이미지 데이터에 대한 개별 픽셀 값의 벡터로 인코딩 된 풍부하고 고차원적인 데이터 세트로 작동합니다. 오디오 처리 시스템 또한 오디오 데이터에 대한 파워 스펙트럼 밀도 계수를 데이터로 사용합니다. 이미지 또는 음성 인식과 같은 작업에서는 작업을 성공적으로 수행하는 데 필요한 모든 정보가 데이터에 인코딩 되어있다는 것을 알고 있습니다 그러나 자연 언어 처리 시스템은 전통적으로 단어를 개별 원자 기호로 취급하므로, 가령 '고양이'는 $Id537$로, '개'는 $Id143$로 표시 됩니다.

<img src="https://www.tensorflow.org/images/audio-image-text.png" width="1000">

이러한 인코딩은 임의적이며 개별 기호 사이에 존재하는 관련성에 대한 유용한 정보를 시스템에 제공하지 않습니다. 이것은 모델이 '개'에 대한 데이터를 처리 할 때 '고양이'에 대해 배운 것(예: 동물, 4개의 다리, 애완 동물 등)을 거의 활용할 수 없다는 것을 의미합니다. 단어를 고유한 이산 ID로 표시하면 데이터 희소성이 생기며 일반적으로 통계 모델을 성공적으로 훈련하기 위해 더 많은 데이터가 필요할 수 있습니다. 벡터 표현을 사용하면 이러한 문제 중 일부를 해결 할 수 있습니다.

[Vector space models](https://en.wikipedia.org/wiki/Vector_space_model) (VSM)은 연속 벡터 공간에서 의미적으로 유사한 단어를 서로 가까운 위치에 매핑되게 임베딩합니다. VSMs는 긴 역사를 가지고 있지만, 모든 방법은 단어들이 어떤 의미를 공유하는 지를 측정하는 [Distributional Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics#Distributional_Hypothesis)에 기반을 둡니다. 이 원리를 활용하는 다양한 접근 방식에는 Count-based법(Latent Semantic Analysis)과 예측 방법([neural probabilistic language models](http://www.scholarpedia.org/article/Neural_net_language_models))이 있습니다.  

이 방법은 다음 논문([Baroni et al.](http://clic.cimec.unitn.it/marco/publications/acl2014/baroni-etal-countpredict-acl2014.pdf))에 보다 자세히 설명되어 있습니다. 요약하면:Count-based법은 큰 단어 코퍼스에서 이웃 단어와 함께 얼마나 자주 단어가 발생하는지 통계를 계산 한 다음 각 단어에 대해 작고 밀도가 높은 벡터를 계산합니다. 예측 모델은 학습 된 작고 밀도가 높은 벡터 (모델의 매개 변수로 간주)의 관점에서 주변 단어를 직접 예측하려고합니다.

word2vec은 Raw 텍스트에서 word embedding을 학습하기위한 고효율 예측 모델입니다. 그것은 두 가지 방법, 즉 Continuous Bag-of-words(CBOW) 모델 과 Skip-Gram 모델 (Mikolov et al.의 3.1 및 3.2 절)이 있습니다. 알고리즘 적으로, CBOW는 소스 컨텍스트 단어 ('고양이가 앉아있는')에서 대상 단어('mat')를 예측하고 Skip-Gram은 역함수를 수행하여 대상에서 소스 컨텍스트 단어를 예측한다는 점에서 비슷합니다.


이러한 역함수는 임의적인 것 처럼 보일 수 있지만, 통계적으로 볼 때, CBOW는 전체 컨텍스트를 하나의 관찰로 처리하여, 많은 정보에 대해 부드럽게 처리합니다. 그리고, 대부분 데이터 세트가 작은 경우 유용합니다. 그러나 skip-gram은 각 컨텍스트-타겟 쌍을 새로운 관측 값으로 처리하므로, 데이터 집합이 큰 경우 더 잘 수행됩니다. 이 튜토리얼의 나머지 부분에서는 skip-gram 모델에 초점을 맞출 것이다.


## Scaling up with Noise-Contrastive Training
신경 확률론적 언어 모델은 이전 단어 $h$가 [소프트 맥스](https://en.wikipedia.org/wiki/Softmax_function) 함수로 주어졌을 때, 다음 단어 $w_t$의 확률을 최대화하기 위해 ML(maximum likelihood)를 사용하여 학습했습니다. 
\begin{align}
P(w_t | h) &= \text{softmax} (\text{score} (w_t, h)) \\
           &= \frac{\exp \{ \text{score} (w_t, h) \} }
             {\sum_\text{Word w' in Vocab} \exp \{ \text{score} (w', h) \} }
\end{align}
여기서 $score(w_t,h)$는 단어 $w_t$와 context $h$의 호환성을 계산합니다. 우리는 학습 세트에 대한 log-likelihood를 최대화하는 방법을 통해 학습합니다. 
\begin{align}
 J_\text{ML} &= \log P(w_t | h) \\
  &= \text{score} (w_t, h) -
     \log \left( \sum_\text{Word w' in Vocab} \exp \{ \text{score} (w', h) \} \right).
\end{align}
이는 언어 모델링에 맞게 정규화 된 확률 모델을 산출합니다. 그러나 이것은 매우 많은 연산을 필요로 합니다. 왜냐하면 모든 학습 단계에서 현재 상황 h에서 w'에 대한 모든 다른 단어 V의 점수를 사용하여 각 확률을 계산하고 정규화해야하기 때문입니다.
<img src="https://www.tensorflow.org/images/softmax-nplm.png" width="600">
반면에 word2vec에서는 피쳐 학습을 위해 완전한 확률 모델이 필요하지 않습니다. 대신에 CBOW와 skip-gram 모델을 이진 분류 목적 (로지스틱 회귀)을 사용하여 $k$로 부터 실제 목표 단어 $w_t$를 가상 단어(노이즈) $\tilde w$와 동일한 문맥에서 구별합니다. 아래에서 CBOW 모델에 대해 설명합니다. skip-gram의 경우 간단히 방향만 반전됩니다.
<img src="https://www.tensorflow.org/images/nce-nplm.png" width="600">
수학적으로, (각 예제에 대한) 목적함수를 최대화 합니다. 
\begin{align}
J_\text{NEG} = \log Q_\theta(D=1 |w_t, h) +
  k \mathop{\mathbb{E}}_{\tilde w \sim P_\text{noise}}
     \left[ \log Q_\theta(D = 0 |\tilde w, h) \right]
\end{align}
여기서 $Q_\theta (D = 1 | w, h)$는 학습 된 관점에서 계산 된 데이터 집합 $D$의 문맥 $h$에서 단어 $w$를 보는 모델에서의 binary logistic regression probability입니다. 이것은 학습된 embedding vectors로 부터 계산됩니다. 실제로 우리는 노이즈 분포로부터 $k$ 대조 단어를 그려서 기대치를 근사화합니다.(i.e. we compute a [Monte Carlo average](https://en.wikipedia.org/wiki/Monte_Carlo_integration)).

목적함수는 모델이 실제 단어에 높은 확률을 할당하고, 노이즈 단어에 낮은 확률을 할당 할 때 최대화됩니다. 기술적으로 이것을 [네거티브 샘플링](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)이라고 부릅니다. 그리고 여기에는 손실 함수라는 좋은 수학적 방법이 있습니다. 제안 된 업데이트 방법은 softmax 함수의 한계치 내에서 업데이트를 근사합니다. 그러나 계산 기능으로 특히 손실 함수 계산이 이제는 단어백터($V$)의 모든 단어가 아닌 선택된 $k$의 노이즈 단어 수로 만 조정되므로 특히 매력적입니다. 따라서 훨씬 더 빨리 학습됩니다. 우리는 TensorFlow가 제공하는 유사 잡음-대조 추정 (NCE) 손실함수 **tf.nn.nce_loss ()**를 실제로 활용할 것입니다. 실제로 어떻게 동작하는지 직관적인 느낌을 갖도록합시다!


## The Skip-gram Model

데이터 셋을 예로 들어 보겠습니다. 

**the quick brown fox jumped over the lazy dog**

우리는 먼저 단어들의 데이터 세트와 그 단어가 나타나는 문맥(Context)를 형성합니다. 우리는 의미있는 방식으로 '문맥'을 정의 할 수 있습니다. 사실 인간은 타겟의 왼쪽, 혹은 오른쪽단어들을 통해 구문적 문맥을 계속 보고 있습니다.(i.e. 현재 목표 단어에 대한 구문적 의존성 e.g. [Levy et al.](https://levyomer.files.wordpress.com/2014/04/dependency-based-word-embeddings-acl-2014.pdf)), 이제 위 정의에 맞게 왼쪽에서 오른쪽으로 단어의 '컨텍스트'를 정의합시다. 1의 크기를 가지는 아래와 같은 **(context, target)** 짝 데이터 집합을 만듭니다.

**([the, brown], quick), ([quick, fox], brown), ([brown, jumped], fox), ...**

skip-gram은 컨텍스트와 타겟을 뒤집어서, 각 타겟 단어에서 각 컨텍스트 단어를 예측하려고 합니다. 따라서 아래와 같은 **(input, output)** 쌍의 데이터 셋을 만듭니다. 

**(quick, the), (quick, brown), (brown, quick), (brown, fox), ...**

목적 함수는 전체 데이터 세트에 대해 정의되지만, 일반적으로 한 번에 한 가지 예제 (또는 batch_size 예제의 'minibatch', 일반적으로 16 <= batch_size <= 512)를 사용하여 확률적인 그래디언트 디센트 (SGD)로 최적화 합니다. 이제 이 과정을 살펴 보겠습니다.

위의 첫 번째 학습 사례를 통해 학습 단계 $t$에서 상상해보십시오. 목표는 최대한 빨리 예측하는 것입니다. 우리는 어떤 잡음 분포, 전형적으로 unigram 분포 인 $P(w)$로 부터 잡음이 많은 (대조적 인) 예제의 $num_noise$를 설정합니다. 단순함을 위해 ** num_noise = 1 **라고 가정하고, 예로 "sheep"을 선택합니다. 그 다음 observed - noisy 데이터 쌍에 대한 손실함수를 계산합니다. 타임스텝 t에서 목적함수는 아래와 같습니다. 

\begin{align}
J^{(t)}_\text{NEG} = \log Q_\theta(D=1 | \text{the, quick}) +
  \log(Q_\theta(D=0 | \text{sheep, quick}))
\end{align}

목적함수를 최적화 하는 방향으로 embedding parameters $\Theta$를 업데이트 하는게 목적입니다. 이 같은 작업은다음과 같이 embedding parameters $\Theta$에 대한 손실함수의 gradient를 통해 수행 할 수 있습니다.  $\frac{\partial}{\partial \theta} J_\text{NEG}$(다행히도 tensorflow는 이러한 작업을 수행하는 함수를 제공합니다. 그런 다음 그라디언트 방향으로 매 스텝마다  업데이트를 수행합니다. 이 과정이 전체 학습 세트에 대해 반복하면서, 모델은 실제 단어와 noise 단어를 구별하는데 성공할 때까지 각 단어에 대해 주변 벡터를 '이동'시킵니다. 

우리는 학습 된 벡터를 [t-SNE dimensionality reduction technique](http://lvdmaaten.github.io/tsne/)과 같은 것을 사용하여 2 차원으로 투영함으로써 시각화 할 수 있습니다. 우리가 이러한 시각화 통해 분석 때, 벡터가 단어와 단어와의 관계에 대한 일반적이고 실제로 유용한 의미 정보를 포착한다는 것을 명백히 볼 수 있습니다. 벡터 공간의 특정 방향이 흥미로운 특정 의미 관계를 가지는 것을 볼 수 있습니다. 예를 들어 남성 - 여성, 동사 시제, 심지어 단어 사이의 국가-수도 관계가 있습니다. (아래 그림 참조) (Mikolov et al., 2013 참조)

<img src="https://www.tensorflow.org/images/linear-relationships.png" width="1000">

이것은 왜 이러한 벡터가 품사 태깅이나 명명 된 개체 인식과 같은 많은 정규 NLP 예측 작업을 위한 기능으로 유용한지를 설명합니다.(예 : [Collobert et al., 2011 (pdf)](http://arxiv.org/abs/1103.0398), [Turian et al., 2010](http://www.aclweb.org/anthology/P10-1040)).

지금 부터 위 단어 벡터 공간 이미지를 그려 보겠습니다.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [10]:
# Step 1: Download the data.
url = 'http://mattmahoney.net/dc/'

In [11]:
def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.request.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    print(statinfo.st_size)
    raise Exception(
        'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

In [12]:
filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [13]:
# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

In [14]:
words = read_data(filename)
print('Data size', len(words))

Data size 17005207


In [15]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000

In [16]:
def build_dataset(words, vocabulary_size):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

In [17]:
data, count, dictionary, reverse_dictionary = build_dataset(words, vocabulary_size)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5235, 3084, 12, 6, 195, 2, 3137, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [18]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

In [19]:
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

3084 originated -> 5235 anarchism
3084 originated -> 12 as
12 as -> 6 a
12 as -> 3084 originated
6 a -> 195 term
6 a -> 12 as
195 term -> 6 a
195 term -> 2 of


In [20]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

## Building the Graph
embeddings에 관한 것이므로, 먼저 embedding matrix를 정의합니다. 이것은 단지 초기화하기 위한 임의의 행렬입니다. 균일하게 값을 초기화합니다.

```python
embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0)) 
```
noise-contrastive 추정 손실(loss)은 로지스틱 회귀 모델로 정의됩니다. 이를 위해 어휘의 각 단어에 대한 가중치와 바이어스를 정의해야합니다. 입력 가중치와는 반대로 출력 가중치라고도 합니다. 다음과 같이  정의합니다.

```python
nce_weights = tf.Variable(
  tf.truncated_normal([vocabulary_size, embedding_size],
                      stddev=1.0 / math.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
```
이제 매개변수를 가져 왔으므로 skip-gram 모델 그래프를 정의 할 수 있습니다. 간단히하기 위해 각 단어가 정수로 표현되도록 텍스트 말뭉치(corpus)를 정수로 가정합니다. (자세한 내용은 tensorflow / examples / tutorials / word2vec / word2vec_basic.py 참조).  skip-gram 모델은 두 개의 입력을받습니다. 하나는 소스 컨텍스트 단어를 나타내는 정수로 구성된 배치(batch)고, 다른 하나는 타겟 단어입니다. 이 입력에 대한 placeholder 노드를 만들어서 나중에 데이터를 공급할 수 있도록 하겠습니다.

```python
#Placeholders for inputs
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
```
이제 해야 할 일은 배치의 각 입력 단어에 대한 벡터를 찾는 것입니다. TensorFlow는 이 작업을 쉽게 수행 할 수 있는 함수를 제공해 줍니다. 
```python
embed = tf.nn.embedding_lookup(embeddings, train_inputs)
```

이제 각 단어에 대한 임베딩을 수행 했으므로, noise-contrastive training objective를 사용하여 타겟 단어를 예측합니다.

```python
#Compute the NCE loss, using a sample of the negative labels each time.
loss = tf.reduce_mean(
  tf.nn.nce_loss(weights=nce_weights,
                 biases=nce_biases,
                 labels=train_labels,
                 inputs=embed,
                 num_sampled=num_sampled,
                 num_classes=vocabulary_size))
```

이제 우리는 Loss 노드가 있으므로, 그라디언트를 계산하고 매개 변수를 업데이트하는 데 필요한 노드를 추가해야합니다.이를 위해 stochastic 그래디언트 디센트를 사용할 것이고 TensorFlow는 이것을 쉽게 만들 수 있는 편리한 함수가 있습니다.

```python
#We use the SGD optimizer.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)
```


In [1]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()

NameError: name 'tf' is not defined

## Training the Model
Training the model is then as simple as using a **feed_dict** to push data into the placeholders and calling tf.Session.run with this new data in a loop.

그런 다음 ** feed_dict **를 사용하여 데이터를 placeholder에 넣고 새 데이터로 tf.Session.run을 반복 호출하는 것으로 간단히 학습을 진행 할 수 있습니다. 

```python
for inputs, labels in generate_batch(...):
  feed_dict = {train_inputs: inputs, train_labels: labels}
  _, cur_loss = session.run([optimizer, loss], feed_dict=feed_dict)
```

전체 코드는 tensorflow/examples/tutorials/word2vec/word2vec_basic.py 를 참조하길 바랍니다. 

In [22]:
# Step 5: Begin training.
num_steps = 10001 #100001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()


Initialized
Average loss at step  0 :  288.126434326
Nearest to seven: auditor, henryk, cusco, gst, perpendicular, obeying, gayle, ulrike,
Nearest to only: godavari, semitism, blogs, tetra, leh, dogs, overboard, resonant,
Nearest to while: pasting, spellings, halt, ralph, trusteeship, platt, accumulator, mon,
Nearest to and: meticulous, differentials, wtro, stoics, depicts, assigned, molybdenum, miniscule,
Nearest to such: finnegans, parnassus, andropov, biddle, appreciated, staunchly, longships, mammalia,
Nearest to s: zambian, pmid, bachman, exasperation, misdiagnosed, mim, dumplings, reduces,
Nearest to be: disparagingly, deuce, infractions, microorganisms, ritz, spy, ici, dashiell,
Nearest to an: physicians, khoi, raisins, telstra, sudden, inept, confounding, apollonia,
Nearest to other: reopening, windmills, rooney, li, nw, trumped, hevelius, rites,
Nearest to three: tdma, errico, sukarno, havas, fiedler, tracer, edwina, bhra,
Nearest to at: clocking, sargent, dara, tilden, deepen

In [23]:
# Step 6: Visualize the embeddings.


def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

In [24]:
try:
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels)

except ImportError:
  print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")

## Visualizing the Learned Embeddings

교육이 끝나면 t-SNE를 사용하여 학습 된 임베딩을 시각화 할 수 있습니다.

<img src="https://www.tensorflow.org/images/tsne.png" width="1000">

Et voila! 예상대로 비슷한 단어가 서로 가까이에 클러스터링됩니다. TensorFlow의 고급 기능을 더 많이 보여주는 word2vec를 구현하려면 [tensorflow_models/tutorials/embedding/word2vec.py](https://www.github.com/tensorflow/tensorflow/blob/r1.1/tensorflow_models/tutorials/embedding/word2vec.py)의 구현을 참조하십시오.




## Evaluating Embeddings: Analogical Reasoning

Embedding은 NLP를 사용한 다양한 예측 작업에 유용합니다. full-blown part-of-speech model이나 named-entity model을 학습 할수도 있고, 간단하게는 ** king is to queen as father is to ? **와 같은 의미론적 구문 관계를 예측하는 데 직접 사용 할 수 있습니다.이것을 유추 추론이라 부르며, [Mikolov and colleagues](http://msr-waypoint.com/en-us/um/people/gzweig/Pubs/NAACL2013Regularities.pdf)에서 이 작업을 자세히 다루고 있습니다. 여기에 관한 데이터 셋은 다음 링크에서 다운 받을 수 있습니다. [download.tensorflow.org](http://download.tensorflow.org/data/questions-words.txt).

완성된 모델을 어떻게 평가하는지는 [tensorflow_models/tutorials/embedding/word2vec.py](https://www.github.com/tensorflow/tensorflow/blob/r1.1/tensorflow_models/tutorials/embedding/word2vec.py)의 **build_eval_graph()**와 **eval()** 에 나와 있습니다.  

하이퍼 매개변수를 설정하는 것은 작업의 정확성에 크게 영향을 줄 수 있습니다. 이 작업에서 state-of-the-art 성능을 얻으려면 매우 큰 데이터 세트를 통해 학습해야하며, 하이퍼 매개변수를 신중하게 튜닝하고 데이터를 서브 샘플링하는 등의 트릭을 사용해야합니다. 이는 이 튜토리얼의 범위를 벗어납니다.


##  Optimizing the Implementation


우리의 기본 구현은 TensorFlow의 유연성을 보여줍니다. 목적함수 바꾸는 것을 예를 들면, **tf.nn.sampled_softmax_loss()**와 같은 off-the-shelf를 **tf.nn.nce_loss ()** 함수와 스와핑하는 것으로 간단히 설정할 수 있습니다. 손실 함수에 대한 새로운 아이디어가 있으면 TensorFlow에서 새로운 목표에 대한 수식을 수동으로 작성하고 옵티마이저가 derivative를 계산하게 할 수 있습니다. 이러한 유연성은 여러 가지 아이디어를 시도하고 신속하게 반복하는 머신러닝 모델을 개발하는데 매우 중요합니다.

만족할만한 모델 구조를 갖추었다면, 최적화하여 더 효율적으로 실행하고 더 짧은 시간에 더 많은 데이터를 처리 할 필요가 있습니다. 예를 들어 우리가이 튜토리얼에서 사용한 기본 코드는 데이터 항목을 읽고 넣는데 Python을 사용하기 때문에 속도가 느립니다. 데이터 입력에 심한 병목현상이 발생될 경우, 커스텀 데이러 리더를 구현하길 원할 수 있습니다. 이 같은 방법은 [New Data Formats](https://www.tensorflow.org/extend/new_data_formats)에 소개되어 있습니다. Skip-Gram modeling의 경우 이 부분은 이미 구현되어 있습니다. [tensorflow_models/tutorials/embedding/word2vec.py](https://www.github.com/tensorflow/tensorflow/blob/r1.1/tensorflow_models/tutorials/embedding/word2vec.py).

모델이 더 이상 입출력에 영향을 받지 않지만, 더 성능을 높이길 원할 경우 [Adding a New Op](https://www.tensorflow.org/extend/adding_an_op)에서 설명한대로 커스텀 TensorFlow Ops을 작성할 수 있습니다. 이 부분 역시 Skip-Gram 예제에 나와 있습니다.[tensorflow_models/tutorials/embedding/word2vec_optimized.py](https://www.github.com/tensorflow/tensorflow/blob/r1.1/tensorflow_models/tutorials/embedding/word2vec.py). 예제를 벤치마킹하여 각 단계에서 향상된 성능을 확인하길바랍니다. 

## Conclusion

이 튜토리얼에서는 word embeddings을 학습하기 위한 효율적인 모델인 word2vec 모델에 대해 설명했습니다. embedding이 왜 유용하고 효율적인지 살펴 봤습니다. 그리고 TensorFlow에서 이 모든 것을 구현하는 방법을 알아봤습니다. 끝으로, 이 튜토리얼이 TensorFlow의 초기 개발에 필요한 유연성과 세부 설정에 필요한 제어 기능을 어떻게 다루는 지를 보여줬길 바랍니다.
